<a href="https://colab.research.google.com/github/prashantiyaramareddy/AI-ML-Learnings/blob/master/ComputerVision/VGGNet_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VGG-Net Implementation

VGGNet is a convolutional neural network architecture proposed by the Visual Geometry Group (VGG) from Oxford University. It is known for its simplicity and uniform architecture, using small convolutional filters and deep layers.

In [1]:
### Import Required Libraries
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
